In [ ]:
import pandas as pd
import numpy as np

#读取文件
sojournTime_raw = pd.read_csv('sojournTime.csv',index_col=0)
comDelay_raw    = pd.read_csv('comDelay.csv',index_col=0)
load_raw     = pd.read_csv('load.csv',index_col=0)
memoryUtil_raw  = pd.read_csv('memoryUtil.csv',index_col=0)

# 得到cloudlet数量
cloudlet_numbers = sojournTime_raw.shape[1]

# 对comDelay和load进行同向化
comDelay_txh = comDelay_raw.apply(lambda x: (np.max(x) - x)) 
load_txh = load_raw.apply(lambda x: (np.max(x) - x)) 

# 合并成一个DataFrame
data = pd.concat([sojournTime_raw,comDelay_txh,load_txh,memoryUtil_raw],axis=1)

# 对所有数据进行归一化
data_nor = data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
data_nor


In [ ]:
# 云模型处理
def cloud_model(df):
    Ex = df.mean()
    En = np.sqrt(np.pi/2)  * np.mean( np.abs(df-Ex) )
    He = np.sqrt(np.abs(df.mean() * df.mean() - df.std() * df.std()))
    return pd.concat([Ex,En,He],axis=1)
data_cloudModel = cloud_model(data_nor)
data_cloudModel

In [ ]:
# 中智聚合

# 中智数
truth = data_cloudModel[0]
indeterminacy = data_cloudModel[1]
falsity = data_cloudModel[2]

# 权重
weight = [0.25,0.25,0.25,0.25]

# 聚合之后的中智数
aggre_truth = []
aggre_indeterminacy = []
aggre_falsity = []

for i in range(cloudlet_numbers):
    tempT = 1
    tempI = 1
    tempF = 1
    # 四个属性
    for j in range(4):
        tempT *= (1 - truth[i+10*j]) ** weight[j]
        tempI *= indeterminacy[i+10*j] ** weight[j]
        tempF *= falsity[i+10*j] ** weight[j]
    aggre_truth.append((1-tempT))
    aggre_indeterminacy.append(tempI)
    aggre_falsity.append(tempF)

# 添加索引
# index = pd.Series(np.arange(0,cloudlet_numbers))
# index = index.astype(str)
# index = 'cloudlet'+index
neutrosophic = pd.DataFrame(np.vstack((aggre_truth,aggre_indeterminacy,aggre_falsity)).T)
neutrosophic


In [ ]:
# 得分函数
score = neutrosophic[0]+1-neutrosophic[1]+1-neutrosophic[2]
print(score)
print('最佳cloudlet为：',score.idxmax())

with open("optimal_cloudlet.txt","w") as f:
        f.write(str(score.idxmin()))  #这句话自带文件关闭功能，不需要再写f.close()
